In [27]:
# zipfile used to extract training and test images

import zipfile

In [28]:
#import zipfile
#with zipfile.ZipFile("150253_347412_bundle_archive.zip", 'r') as zip_ref:
#    zip_ref.extractall("Currency_Images")

In [29]:
import tensorflow as tf

In [30]:
print(tf.__version__)

2.3.0


In [31]:
# module used to generate custom variations on the images

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [32]:
# variations on the training set images

train_data_gen = ImageDataGenerator(
    rescale=1./255.0, 
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
    )


In [33]:
# extract training images from directory

training_set = train_data_gen.flow_from_directory(
                directory = "Currency_Images/coins/data/train",
                target_size = (64, 64),
                class_mode = 'categorical',
                batch_size = 32,
                subset = 'training'
                )

Found 6413 images belonging to 211 classes.


In [34]:
# variations on validation set images : rescaling

valid_data_gen = ImageDataGenerator(
    rescale=1./255.0
    )


In [35]:
# validation set images extracted from directory

validation_set = valid_data_gen.flow_from_directory(
        directory = 'Currency_Images/coins/data/validation',
        target_size=(64, 64),
        class_mode='categorical',
        batch_size=32
)

Found 844 images belonging to 211 classes.


In [36]:
# variations on the test images : rescaling

test_data_gen = ImageDataGenerator(rescale=1./255.0)

In [37]:
# test images extracted from directory

test_set = test_data_gen.flow_from_directory(
        directory = 'Currency_Images/coins/data/test',
        target_size=(64, 64),
        class_mode='categorical',
        batch_size=32
)

Found 844 images belonging to 211 classes.


In [38]:
# imports for the custom model

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D,MaxPool2D,Dropout,Flatten,Dense

In [39]:
# initiating the model
custom_model = Sequential()

# first set of convolution and pooling operations along with dropout
custom_model.add(Conv2D(16,(3,3),activation='relu',input_shape=(64,64,3)))
custom_model.add(MaxPool2D(2,2))
custom_model.add(Dropout(0.2))

# second set of convolution and pooling operations along with dropout
custom_model.add(Conv2D(32,(3,3),activation='relu'))
custom_model.add(MaxPool2D(2,2))
custom_model.add(Dropout(0.3))

# third set of convolution and pooling operations along with dropout
custom_model.add(Conv2D(64,(3,3),activation='relu'))
custom_model.add(MaxPool2D(2,2))
custom_model.add(Dropout(0.3))

# flattening the output from previous layers
custom_model.add(Flatten())

# first dense layer followed by a 0.2 dropout layer
custom_model.add(Dense(256,activation='relu'))
custom_model.add(Dropout(0.2))

# second dense layer followed by a 0.2 dropout layer
custom_model.add(Dense(256,activation='relu'))
custom_model.add(Dropout(0.2))

# third dense layer followed by a 0.2 dropout layer
custom_model.add(Dense(256,activation='relu'))
custom_model.add(Dropout(0.2))

# fourth dense layer followed by a 0.2 dropout layer
custom_model.add(Dense(256,activation='relu'))
custom_model.add(Dropout(0.2))

# output layer
custom_model.add(Dense(211,activation='softmax'))

In [40]:
# compile the model

custom_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [41]:
custom_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 16)        448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 16)        0         
_________________________________________________________________
dropout (Dropout)            (None, 31, 31, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 64)        1

In [42]:
# model fit on the training set

history_custom_model = custom_model.fit_generator(generator = training_set,
                            validation_data = validation_set,
                            epochs =20
                            )

Epoch 1/20
201/201 [==============================] - 51s 256ms/step - loss: 5.3482 - accuracy: 0.0061 - val_loss: 5.3499 - val_accuracy: 0.0059
Epoch 2/20
201/201 [==============================] - 105s 524ms/step - loss: 5.2182 - accuracy: 0.0097 - val_loss: 5.1135 - val_accuracy: 0.0095
Epoch 3/20
201/201 [==============================] - 112s 557ms/step - loss: 4.9639 - accuracy: 0.0156 - val_loss: 4.9700 - val_accuracy: 0.0095
Epoch 4/20
201/201 [==============================] - 106s 529ms/step - loss: 4.9055 - accuracy: 0.0187 - val_loss: 4.9107 - val_accuracy: 0.0083
Epoch 5/20
201/201 [==============================] - 114s 568ms/step - loss: 4.8035 - accuracy: 0.0268 - val_loss: 4.7846 - val_accuracy: 0.0273
Epoch 6/20
201/201 [==============================] - 121s 601ms/step - loss: 4.7063 - accuracy: 0.0395 - val_loss: 4.6968 - val_accuracy: 0.0284
Epoch 7/20
201/201 [==============================] - 209s 1s/step - loss: 4.6033 - accuracy: 0.0457 - val_loss: 4.7145 - val

In [43]:
# final evaluation on the test set

score_custom_model = custom_model.evaluate_generator(test_set, verbose=2)


27/27 - 8s - loss: 4.1749 - accuracy: 0.1173


In [44]:
# test loss and test accuracy

print('Custom Model : Test Loss is : ', score_custom_model[0])
print('Custom Model : Test Accuracy is : ', score_custom_model[1])

Custom Model : Test Loss is :  4.1748528480529785
Custom Model : Test Accuracy is :  0.11729858070611954


In [ ]:
custom_model.save("Custom_model_for_currency_classification.h5")